## Imports & helper functions

In [36]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from timeit import default_timer as timer
from tqdm.notebook import tqdm
from torchinfo import summary
from pathlib import Path
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from ptflops import get_model_complexity_info
import psutil
from model import SmokersBinaryClassification
from utils import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [37]:
# Calculate accuracy
def accuracy_fn(y_pred, y_true):
    # print(f'prediction {y_pred} true: {y_true} ')
    correct = torch.eq(y_true, y_pred).sum().item()
    return (correct / len(y_pred)) * 100

In [38]:
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.
    """
    total_time = end - start
    print(f'Train time on {device}: {total_time:.3f} seconds')
    # return total_time

In [39]:
def training_loop(model, X_train, y_train, X_test, y_test, epochs, loss_fn, optimizer):
    X_train, y_train, X_test, y_test = X_train.to(device), y_train.to(device), X_test.to(device), y_test.to(device)

    start_time = timer()

    for epoch in tqdm(range(epochs), desc='Training...'):
        model.train()

        # 1. Forward passs
        y_hat = model(X_train)
        loss = loss_fn(y_hat, y_train)    

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate prediction accuracy
        y_train_pred = torch.round(torch.sigmoid(y_hat))        #if not using sigmoid on model output
        # y_train_pred = torch.round(y_hat)                     #prediction probabilities -> prediction labels
        acc = accuracy_fn(y_pred=y_train_pred, y_true=y_train)

        ### Testing
        model.eval()
        with torch.inference_mode():
            y_hat = model(X_test)
            test_loss = loss_fn(y_hat, y_test)

            y_test_pred = torch.round(torch.sigmoid(y_hat))
            # y_test_pred = torch.round(y_hat)
            test_acc = accuracy_fn(y_pred=y_test_pred, y_true=y_test)

        if epoch % 50 == 0 and epoch != 0             :
            print(f'Epoch: {epoch:04d} | Loss: {loss:.4f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')

    end_time = timer()
    return end_time - start_time, acc, test_acc
  

1. Initialize model
2. Save parameter values
3. Train network
4. Prune
5. Restore values from step 2 to pruned network
6. Repeat steps 3 - 5

## Data load and prep

In [40]:
dataset = pd.read_csv('dataset/smoking_driking_dataset_Ver01.csv')

In [41]:
dataset = dataset[dataset['SMK_stat_type_cd'] != 2]

In [42]:
dataset['sex'] = dataset['sex'].replace('Male', 0.0)
dataset['sex'] = dataset['sex'].replace('Female', 1.0)

In [43]:
dataset['SMK_stat_type_cd'] = dataset['SMK_stat_type_cd'].apply(lambda x: 1.0 if x == 3.0 else 0.0) # Smokers: 1 non smokers: 0

In [44]:
# Remove some input features from dataset
dataset = dataset.drop(['waistline','sight_left','sight_right','hear_left','hear_right','urine_protein','DRK_YN'], axis=1)

In [45]:
print(f'Number of smokers in the dataset: {len(dataset[dataset["SMK_stat_type_cd"]==1])}')
print(f'Number of non-smokers in the dataset: {len(dataset[dataset["SMK_stat_type_cd"]==0])}')

Number of smokers in the dataset: 213954
Number of non-smokers in the dataset: 602441


In [46]:
print(f'Number of smokers in the dataset: {len(dataset[dataset["SMK_stat_type_cd"]==1])}')
print(f'Number of non-smokers in the dataset: {len(dataset[dataset["SMK_stat_type_cd"]==0])}')

Number of smokers in the dataset: 213954
Number of non-smokers in the dataset: 602441


In [47]:
# Construct pandas dataframes for X and y values
df_X = dataset.drop(['SMK_stat_type_cd'], axis=1)
df_y = pd.DataFrame(dataset['SMK_stat_type_cd'])

In [48]:
# Train Test Split, Set X, y
X_train, X_test, y_train, y_test = train_test_split(df_X.to_numpy(), df_y.to_numpy(), test_size=0.2, random_state=1)

# Convert X features to float tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

# Conver y labels to float tensors
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

## Training

In [49]:
model = SmokersBinaryClassification(input_features=16).to(device)

In [50]:
original_params = save_parameters(model)

In [51]:
# loss_fn = torch.nn.BCELoss().to(device)
loss_fn = torch.nn.BCEWithLogitsLoss().to(device)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer= torch.optim.Adam(model.parameters(), lr=0.01)

In [52]:
for i in range(4):
    print(summary(model))
    time, train_acc, test_acc = training_loop(model, X_train, y_train, X_test, y_test, 400, loss_fn, optimizer)
    print(f'Iteration {i} training accuracy: {train_acc} | testing accuracy: {test_acc}')
    prune.l1_unstructured(model.stack[0], name='weight', amount=0.2)
    prune.l1_unstructured(model.stack[2], name='weight', amount=0.2)   
    restore_original_parameters(original_params, model.state_dict())

Layer (type:depth-idx)                   Param #
SmokersBinaryClassification              --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       1,088
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       4,160
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       65
Total params: 5,313
Trainable params: 5,313
Non-trainable params: 0


Training...:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch: 0050 | Loss: 0.4876, Acc: 76.40% | Test Loss: 0.4786, Test Acc: 77.08%
Epoch: 0100 | Loss: 0.4375, Acc: 78.67% | Test Loss: 0.4336, Test Acc: 78.85%
Epoch: 0150 | Loss: 0.4228, Acc: 79.26% | Test Loss: 0.4181, Test Acc: 79.58%
Epoch: 0200 | Loss: 0.3959, Acc: 80.83% | Test Loss: 0.3905, Test Acc: 80.95%
Epoch: 0250 | Loss: 0.3732, Acc: 81.64% | Test Loss: 0.3761, Test Acc: 81.52%
Epoch: 0300 | Loss: 0.3673, Acc: 81.92% | Test Loss: 0.3638, Test Acc: 81.98%
Epoch: 0350 | Loss: 0.3659, Acc: 81.76% | Test Loss: 0.3735, Test Acc: 81.28%
Iteration 0 training accuracy: 82.2863013614733 | testing accuracy: 82.3663790199597
Layer (type:depth-idx)                   Param #
SmokersBinaryClassification              --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       883
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       3,341
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       65
Total pa

Training...:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch: 0050 | Loss: 0.3593, Acc: 82.28% | Test Loss: 0.3573, Test Acc: 82.32%
Epoch: 0100 | Loss: 0.3571, Acc: 82.41% | Test Loss: 0.3551, Test Acc: 82.41%
Epoch: 0150 | Loss: 0.3562, Acc: 82.48% | Test Loss: 0.3545, Test Acc: 82.46%
Epoch: 0200 | Loss: 0.3556, Acc: 82.52% | Test Loss: 0.3541, Test Acc: 82.49%
Epoch: 0250 | Loss: 0.3551, Acc: 82.55% | Test Loss: 0.3538, Test Acc: 82.51%
Epoch: 0300 | Loss: 0.3555, Acc: 82.50% | Test Loss: 0.3543, Test Acc: 82.52%
Epoch: 0350 | Loss: 0.3544, Acc: 82.61% | Test Loss: 0.3533, Test Acc: 82.61%
Iteration 1 training accuracy: 82.50494552269429 | testing accuracy: 82.55317585237538
Layer (type:depth-idx)                   Param #
SmokersBinaryClassification              --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       719
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       2,686
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       65
Total 

Training...:   0%|          | 0/400 [00:00<?, ?it/s]

Epoch: 0050 | Loss: 0.3604, Acc: 82.20% | Test Loss: 0.3580, Test Acc: 82.25%
Epoch: 0100 | Loss: 0.3576, Acc: 82.39% | Test Loss: 0.3556, Test Acc: 82.41%
Epoch: 0150 | Loss: 0.3569, Acc: 82.48% | Test Loss: 0.3549, Test Acc: 82.46%
Epoch: 0200 | Loss: 0.3556, Acc: 82.54% | Test Loss: 0.3537, Test Acc: 82.55%
Epoch: 0250 | Loss: 0.3552, Acc: 82.58% | Test Loss: 0.3537, Test Acc: 82.54%
Epoch: 0300 | Loss: 0.3553, Acc: 82.58% | Test Loss: 0.3536, Test Acc: 82.57%


KeyboardInterrupt: 

In [18]:
summary(model)

Layer (type:depth-idx)                   Param #
SmokersBinaryClassification              --
├─Sequential: 1-1                        --
│    └─Linear: 2-1                       483
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       1,742
│    └─ReLU: 2-4                         --
│    └─Linear: 2-5                       65
Total params: 2,290
Trainable params: 2,290
Non-trainable params: 0